# Quick start: SATRAP CTI analysis toolbox

Import the SATRAP CTI analysis toolbox.

In [3]:
from satrap.service.satrap_analysis import CTIanalysisToolbox
from satrap.settings import TYPEDB_SERVER_ADDRESS, DB_NAME

satrap = CTIanalysisToolbox(TYPEDB_SERVER_ADDRESS, DB_NAME)


## Starting with simple functions

Get the number of STIX Domain Objects (SDOs) in the knowledge base per STIX type.

In [4]:
print(satrap.get_sdo_stats())

+-----------------------+------------------------+
| STIX Domain Objects   |   Total in the CTI SKB |
+=======================+========================+
| intrusion-set         |                    170 |
+-----------------------+------------------------+
| attack-pattern        |                    668 |
+-----------------------+------------------------+
| identity              |                      1 |
+-----------------------+------------------------+
| course-of-action      |                    285 |
+-----------------------+------------------------+
| campaign              |                     34 |
+-----------------------+------------------------+
| tool                  |                     89 |
+-----------------------+------------------------+
| malware-family        |                    624 |
+-----------------------+------------------------+


Print the catalog of MITRE ATT&CK techniques and subtechniques. Subtechniques can be optionally filtered out.

In [5]:
# print(satrap.mitre_attack_techniques(subtechniques=False))
print(satrap.mitre_attack_techniques())


+----------------+--------------------------------------------------------+
| MITRE ATT&CK   | Name                                                   |
| technique ID   |                                                        |
+================+========================================================+
| T1001          | Data Obfuscation                                       |
+----------------+--------------------------------------------------------+
| T1001.001      | Junk Data                                              |
+----------------+--------------------------------------------------------+
| T1001.002      | Steganography                                          |
+----------------+--------------------------------------------------------+
| T1001.003      | Protocol or Service Impersonation                      |
+----------------+--------------------------------------------------------+
| T1003          | OS Credential Dumping                                  |
+-----------

Print the MITRE ATT&CK catalog of mitigations.

In [6]:
print(satrap.mitre_attack_mitigations())

+-----------------+-----------------------------------------+
| MITRE ATT&CK    | Name                                    |
| Mitigation ID   |                                         |
+=================+=========================================+
| M1013           | Application Developer Guidance          |
+-----------------+-----------------------------------------+
| M1015           | Active Directory Configuration          |
+-----------------+-----------------------------------------+
| M1016           | Vulnerability Scanning                  |
+-----------------+-----------------------------------------+
| M1017           | User Training                           |
+-----------------+-----------------------------------------+
| M1018           | User Account Management                 |
+-----------------+-----------------------------------------+
| M1019           | Threat Intelligence Program             |
+-----------------+-----------------------------------------+
| M1020 

Get information on a specific MITRE ATT&CK element (technique, group, software, etc.) using its MITRE ATT&CK id.

In [7]:
try:
	t1 = satrap.get_attck_concept_info("T1027.001")
	for data in t1:
		print(CTIanalysisToolbox.tabulate_stix_obj(data))
except ValueError as exc:
	print(exc)

+---------------------+---------------------------------------------------------+
| Property            | Value                                                   |
+=====================+=========================================================+
| type                | attack-pattern                                          |
+---------------------+---------------------------------------------------------+
| created             | 2020-02-05 14:04                                        |
+---------------------+---------------------------------------------------------+
| stix-id             | attack-pattern--5bfccc3f-2326-4112-86cc-c1ece9d8a2b5    |
+---------------------+---------------------------------------------------------+
| modified            | 2023-03-30 21:01                                        |
+---------------------+---------------------------------------------------------+
| spec-version        | 2.1                                                     |
+---------------

Get information about a STIX object using its STIX id.

In [8]:
try:
	sdo = satrap.search_stix_object("campaign--0c259854-4044-4f6c-ac49-118d484b3e3b")
	print(CTIanalysisToolbox.tabulate_stix_obj(sdo))
except ValueError as e:
	print(e)

+---------------------+---------------------------------------------------------+
| Property            | Value                                                   |
+=====================+=========================================================+
| type                | campaign                                                |
+---------------------+---------------------------------------------------------+
| last-seen           | 2024-01-01 05:00                                        |
+---------------------+---------------------------------------------------------+
| created             | 2024-06-10 18:57                                        |
+---------------------+---------------------------------------------------------+
| alias               | KV Botnet Activity                                      |
+---------------------+---------------------------------------------------------+
| first-seen          | 2022-10-01 04:00                                        |
+---------------

Retrieve mitigations explicitly associated to a specific technique using its STIX id.

In [9]:
mitigations = satrap.mitigations_for_technique("attack-pattern--d1fcf083-a721-4223-aedf-bf8960798d62")
display(mitigations)

{'course-of-action--47e0e9fe-96ce-4f65-8bb1-8be1feacb5db': 'Execution Prevention'}

## CTI analysis through automated reasoning

The following functions use the inference engine to draw relations on the data explicitly stored in the knowlegde base. For some of them, the use of inference is optional.

Get statistics on the usage of ATT&CK techniques by groups. The output of this function is the same as when running the command `satrap techniques` on the CLI.

In [10]:
# Run without the use of the inference engine
try:
	print("( MITRE ATT&CK technique, Name, Used by (num. intrusion sets) )")
	display(satrap.techniques_usage())
except Exception as err:
	print(err)

( MITRE ATT&CK technique, Name, Used by (num. intrusion sets) )


[('T1105', 'Remote File Copy Mitigation', 88),
 ('T1082', 'System Information Discovery Mitigation', 58),
 ('T1083', 'File and Directory Discovery Mitigation', 51),
 ('T1005', 'Data from Local System Mitigation', 48),
 ('T1078', 'Valid Accounts Mitigation', 45),
 ('T1016', 'System Network Configuration Discovery Mitigation', 41),
 ('T1057', 'Process Discovery Mitigation', 41),
 ('T1033', 'System Owner/User Discovery Mitigation', 40),
 ('T1203', 'Exploitation for Client Execution Mitigation', 39),
 ('T1018', 'Remote System Discovery Mitigation', 39),
 ('T1190', 'Exploit Public-Facing Application Mitigation', 38),
 ('T1047', 'Windows Management Instrumentation Mitigation', 38),
 ('T1140', 'Deobfuscate/Decode Files or Information Mitigation', 37),
 ('T1189', 'Drive-by Compromise Mitigation', 32),
 ('T1049', 'System Network Connections Discovery Mitigation', 31),
 ('T1046', 'Network Service Scanning Mitigation', 30),
 ('T1133', 'External Remote Services Mitigation', 30),
 ('T1027', 'Obfusc

In [11]:
# Run using the inference engine
try:
	print("( MITRE ATT&CK technique, Name, Used by (num. intrusion sets) )")
	display(satrap.techniques_usage(infer=True))
except Exception as err:
	print(err)

( MITRE ATT&CK technique, Name, Used by (num. intrusion sets) )


[('T1105', 'Remote File Copy Mitigation', 143),
 ('T1082', 'System Information Discovery Mitigation', 141),
 ('T1083', 'File and Directory Discovery Mitigation', 140),
 ('T1057', 'Process Discovery Mitigation', 129),
 ('T1016', 'System Network Configuration Discovery Mitigation', 118),
 ('T1005', 'Data from Local System Mitigation', 115),
 ('T1140', 'Deobfuscate/Decode Files or Information Mitigation', 110),
 ('T1033', 'System Owner/User Discovery Mitigation', 110),
 ('T1112', 'Modify Registry Mitigation', 104),
 ('T1106', 'Execution through API Mitigation', 101),
 ('T1027', 'Obfuscated Files or Information Mitigation', 100),
 ('T1113', 'Screen Capture Mitigation', 99),
 ('T1041', 'Exfiltration Over Command and Control Channel Mitigation', 92),
 ('T1012', 'Query Registry Mitigation', 89),
 ('T1049', 'System Network Connections Discovery Mitigation', 85),
 ('T1095', 'Standard Non-Application Layer Protocol Mitigation', 84),
 ('T1135', 'Network Share Discovery Mitigation', 81),
 ('T1018'

Get the techniques used by a specific group. Optionally enable inference.

In [12]:
techniques = satrap.techniques_used_by_group("G0025", infer=True)
display(techniques)

{'attack-pattern--be055942-6e63-49d7-9fa1-9cb7d8a8f3f4': 'Bidirectional Communication',
 'attack-pattern--f7827069-0bf2-4764-af4f-23fae0d181b7': 'Dead Drop Resolver',
 'attack-pattern--cdfc5f0a-9bb9-4352-b896-553cfa2d8fd8': 'Establish Accounts',
 'attack-pattern--d63a3fb8-9452-4e9d-a60a-54be68d5998c': 'File Deletion',
 'attack-pattern--7bc57495-ea59-4380-be31-a64af124ef18': 'File and Directory Discovery',
 'attack-pattern--84e02621-8fdf-470f-bd58-993bb6a89d91': 'Multi-Stage Channels',
 'attack-pattern--8f4a33ec-8b1f-4b80-a2f6-642b2e479580': 'Process Discovery',
 'attack-pattern--88d31120-5bc7-4ce3-a9c0-7cf147be8e54': 'Web Services',
 'attack-pattern--d1fcf083-a721-4223-aedf-bf8960798d62': 'Windows Command Shell'}

Obtain the set of courses of action found to mitigate any of the techniques used by a specific group.

In [13]:
display(satrap.related_mitigations(group_name="BlackTech"))

{'course-of-action--b5dbb4c5-b0b1-40b1-80b6-e9e84ab90067': 'Software Configuration',
 'course-of-action--78bb71be-92b4-46de-acd6-5f998fedf1cc': 'Pre-compromise',
 'course-of-action--12241367-a8b7-49b4-b86e-2236901ba50c': 'Network Intrusion Prevention',
 'course-of-action--eb88d97c-32f1-40be-80f0-d61a4b0b4b31': 'Disable or Remove Feature or Program',
 'course-of-action--2a4f6c11-a4a7-4cb9-b0ef-6ae1bb3a718a': 'User Training',
 'course-of-action--21da4fd4-27ad-4e9c-b93d-0b9b14d02c96': 'Restrict Web-Based Content',
 'course-of-action--86598de0-b347-4928-9eb0-0acbfc21908c': 'Network Segmentation',
 'course-of-action--cc2399fd-3cd3-4319-8d0a-fbd6420cdaf8': 'Audit',
 'course-of-action--47e0e9fe-96ce-4f65-8bb1-8be1feacb5db': 'Execution Prevention',
 'course-of-action--b9f0c069-abbe-4a07-a245-2481219a1463': 'Application Isolation and Sandboxing',
 'course-of-action--93e7968a-9074-4eac-8ae9-9f5200ec3317': 'User Account Management',
 'course-of-action--20f6a9df-37c4-4e20-9e47-025983b1b39d': 'Filt

### Explanation of inferred knowledge

**Hint:** The outcome of an explanation includes the underlying query that was executed by the function. For an interactive visual explanation, we suggest to copy and run the query in TypeDB Studio as explained in the [user manual](/docs/manual/interfaces.md).

Obtain an explanation for why (and if) a given mitigation is found to address the techniques used by a group. This function can be used to explain individual courses of action in the output of `related_mitigations` shown above.

For instance, to know why "Remote Data Storage" is found to be a course of action against "BlackTech":

In [14]:
try:
	rel_explanation = satrap.explain_if_related_mitigation("G0098", "course-of-action--20a2baeb-98c2-4901-bad7-dc62d0a03dea")
	print(f"EXECUTED QUERY:\n{rel_explanation.query}\n")
	print("EXPLANATIONS:")
	for tech in rel_explanation.explanations:
		print(CTIanalysisToolbox.format_dict(tech.as_json()), end="\n")
except ValueError as err:
	print(err)

EXECUTED QUERY:
match
$intrusion-set isa intrusion-set, has stix-id 'intrusion-set--6fe8a2a1-a1b0-4af8-953d-4babd329f8f8', has name $group-name;
$course-of-action isa course-of-action, has stix-id 'course-of-action--20a2baeb-98c2-4901-bad7-dc62d0a03dea', has name $mitigation-name;
$r (source-role: $course-of-action, target-role: $intrusion-set) isa related-to;
get;

EXPLANATIONS:
Data inferred for query statement: { $r (source-role:$course-of-action, target-role:$intrusion-set); $r isa related-to; }
Applied rule: course-of-action-for-intrusion-set
Condition met: 
    intrusion-set: (BlackTech, intrusion-set--6fe8a2a1-a1b0-4af8-953d-4babd329f8f8)
    sdo_name: Indicator Removal
    course-of-action: (Remote Data Storage, course-of-action--20a2baeb-98c2-4901-bad7-dc62d0a03dea)
    sdo: (Indicator Removal, attack-pattern--799ace7f-e227-4411-baa0-8868704f2a69)
    use: _Relation[indirectly-uses:0x8470804d7fffffffffffffec] (Inferred)
    mitigation: relationship--60151204-5c0c-4d18-8582-4d0

Obtain relevant courses of action for a given intrusion set, when considering the mitigation of a specific SDO (usually technique).

E.g., What mitigations can we find against "ZIRCONIUM" that specifically address the technique "T1059.006"?

In [15]:
try:
	reason = satrap.explain_related_techniques("ZIRCONIUM", "T1059.006")
	print(f"EXECUTED QUERY\n{reason.query}\n")
	for tech in reason.explanations:
		print(CTIanalysisToolbox.format_dict(tech.as_json()), end="\n")
except Exception as exc:
	print(exc)

EXECUTED QUERY
match
$intrusion-set isa intrusion-set, has name 'ZIRCONIUM';
$sdo isa stix-domain-object, has stix-id 'attack-pattern--cc3502b5-30cc-4473-ad48-42d51a6ef6d1', has name $t_name;
$mitigation (mitigation: $course-of-action, mitigated: $sdo) isa mitigates;
$r (source-role: $course-of-action, target-role: $intrusion-set) isa related-to;
$course-of-action isa course-of-action, has name $name;
get;

Data inferred for query statement: { $r (source-role:$course-of-action, target-role:$intrusion-set); $r isa related-to; }
Applied rule: course-of-action-for-intrusion-set
Condition met: 
    intrusion-set: (ZIRCONIUM, intrusion-set--4283ae19-69c7-4347-a35e-b56f08eb660b)
    sdo_name: Python
    course-of-action: (Audit, course-of-action--cc2399fd-3cd3-4319-8d0a-fbd6420cdaf8)
    sdo: (Python, attack-pattern--cc3502b5-30cc-4473-ad48-42d51a6ef6d1)
    mitigation: relationship--03b2c5b8-aeca-4232-a206-521b83d56c54
    use: relationship--3b689978-c74b-48e6-8d5e-73b3a6a6ea9d
Inferred con

Find out, with an explanation, the set of techniques used by a specific group. Optionally filter to get the explanation of a specific technique.

In [16]:
try:
	dg_explanation = satrap.explain_techniques_used_by_group("G0071", "Domain Groups")
	for tech in dg_explanation.explanations:
		print(CTIanalysisToolbox.format_dict(tech.as_json()), end="\n")
except Exception as err:
	print(err)

Data inferred for query statement: { $usage (used:$technique, user:$group); $usage isa uses; }
Applied rule: transitive-use
Condition met: 
    x: (Orangeworm, intrusion-set--5636b7b3-d99b-4edd-aa05-ee649c1d4ef1)
    name_y: Net
    z: (Domain Groups, attack-pattern--2aed01ad-3df3-4410-a8cb-11ea4ded587c)
    name_z: Domain Groups
    name_x: Orangeworm
    use_yz: relationship--0eb8fe0a-6a6b-4d66-b6ff-e1d7f8f18948
    use_xy: relationship--960d642d-a2f7-4b78-8363-fa94aff7bf86
    y: (Net, tool--03342581-f790-4f03-ba41-e82e67392e23)
Inferred conclusion: 
    indirectly-uses:used: _RoleType[label: uses:used]
    indirectly-uses: _RelationType[label: indirectly-uses]
    x: (Orangeworm, intrusion-set--5636b7b3-d99b-4edd-aa05-ee649c1d4ef1)
    _: _Relation[indirectly-uses:0x8470804d7ffffffffffffffd] (Inferred)
    z: (Domain Groups, attack-pattern--2aed01ad-3df3-4410-a8cb-11ea4ded587c)
    indirectly-uses:indirect-user: _RoleType[label: indirectly-uses:indirect-user]
Mapping of variables (